In [6]:
import os
import pandas as pd
data_dir =  '../cropped/cropped'
data_per = os.listdir(data_dir)

In [105]:
# 1. image을 같은 크기로 resize
# 2. (Data Augmentation)
# 3. list안에 순서대로 들어갈 수 있게

# [[[~,~,~], [~~~],[~~~],[~~],[~~]],[[~~],[~~],[~~~]]]
# x.shape = [image 갯수, frame 갯수, image_size(벡터)]
# y.shape = [image 갯수, ]
# data = np.array(list)
from tensorflow.keras.preprocessing import image
import numpy as np

for person in data_per:
    if person == '.DS_Store':
        continue
    words_dir = os.path.join(data_dir, person) + '/words'
    sentences = os.listdir(words_dir)
    for sentence_id in sentences:
        s_dir = os.path.join(words_dir, sentence_id)
        nums = os.listdir(s_dir)
        x_data = np.empty(3)
        for n in nums:
            frames_dir = os.path.join(s_dir, n)
            frames = os.listdir(frames_dir)
            x_data_ = np.empty(3)
            for frame in frames:
                img_path =f'{frames_dir}/{frame}'
                
                # resize, flatten, nparray
                img = image.load_img(img_path, target_size=(64,64))
                img_array = image.img_to_array(img)
                img_augmented = DataAugmentation().generate(img_array, 10)
                img_tensor = img_augmented.flatten()
                img_tensor /= 255.
                img_tenor = np.expand_dims(img_tensor, axis=0)
                print(img_augmented.shape, img_tenor.shape)
                
                # frame 묶기
                if frame == 'color_001.jpg':
                    x_data_ = img_tensor
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis=0)
                    
                print(x_data_.shape)
                
            # 이미지 통합
            x_data_ = np.expand_dims(x_data_, axis = 0)
            if n == '01':
                x_data = x_data_
            else:
                x_data = np.concatenate([x_data, x_data_], axis=0)
            print(x_data.shape)

(31, 64, 64, 3) (1, 380928)
(380931,)
(31, 64, 64, 3) (1, 380928)
(761859,)
(31, 64, 64, 3) (1, 380928)
(380928,)
(31, 64, 64, 3) (1, 380928)
(761856,)
(31, 64, 64, 3) (1, 380928)
(1142784,)
(31, 64, 64, 3) (1, 380928)
(1523712,)
(31, 64, 64, 3) (1, 380928)
(1904640,)
(31, 64, 64, 3) (1, 380928)
(2285568,)
(31, 64, 64, 3) (1, 380928)
(2666496,)
(31, 64, 64, 3) (1, 380928)
(3047424,)
(31, 64, 64, 3) (1, 380928)
(3428352,)
(31, 64, 64, 3) (1, 380928)
(3809280,)
(31, 64, 64, 3) (1, 380928)
(4190208,)
(31, 64, 64, 3) (1, 380928)
(4571136,)
(31, 64, 64, 3) (1, 380928)
(4952064,)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [104]:
# ref: https://m.blog.naver.com/PostView.nhn?blogId=isu112600&logNo=221582003889&proxyReferer=https:%2F%2Fwww.google.com%2F
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:

#     def __init__(self, img, count):
#         self.data = np.expand_dims(img, axis=0)
#         self.augmentedData = np.empty(3)
#         self.count = count
        

    @classmethod
    def generate(self, img, count):
        self.data = np.expand_dims(img, axis=0)
        self.augmentedData = self.data
        
        self.width_shift_range(self, count)
        self.height_shift_range(self, count)
        self.random_rotation(self, count)
        
        return self.augmentedData
    
    
    # 좌우로 shift
    def width_shift_range(self, count):
        # generatior 생성
        datagen = ImageDataGenerator(width_shift_range=[-20,20])
        
        # iterator 준비
        it = datagen.flow(self.data, batch_size=1)
        
        for i in range(count):
            x_batch = it.next()
            self.augmentedData = np.concatenate([self.augmentedData, x_batch], axis=0)
    
    # 상하로 shift
    def height_shift_range(self, count):
        # generatior 생성
        datagen = ImageDataGenerator(height_shift_range=0.5)
        
        # iterator 준비
        it = datagen.flow(self.data, batch_size=1)
        
        for i in range(count):
            x_batch = it.next()
            self.augmentedData = np.concatenate([self.augmentedData, x_batch], axis=0)
            
    # 랜덤 각도 돌리기
    def random_rotation(self, count):
        # generatior 생성
        datagen = ImageDataGenerator(rotation_range=90)
        
        # iterator 준비
        it = datagen.flow(self.data, batch_size=1)
        
        for i in range(count):
            x_batch = it.next()
            self.augmentedData = np.concatenate([self.augmentedData, x_batch], axis=0)